<a href="https://colab.research.google.com/github/nerudxlf/getting_quartiles_scopus/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

import pandas as pd
from google.colab import files
from collections import Counter
files.upload()

In [11]:
def source_title_update(data: object) -> object:
  data_list = data["Source Title"].to_list()
  result_list = []
  for elem in data_list:
    if isinstance(elem, float):
      result_list.append("not")
      continue
    elem = elem.upper()
    elem = re.sub("[^A-Za-z0-9]", "", elem)
    result_list.append(elem)
  data["KEY"] = result_list
  return data


def get_result(s: object, data: object) -> object:
  result_s_df = pd.merge(left=s, right=data, left_on="KEY", right_on="KEY")
  result_s_df.drop(['Unnamed: 0', 'Source Title_y'], axis=1, inplace=True)
  result_s_df = result_s_df.drop_duplicates(subset=["Title"])
  result_s_df.rename(columns={"Source Title_x": "Source Title"}, inplace=True)
  return result_s_df


def get_n(data: object) -> object:
  data_list = data["Authors with affiliations"].to_list()
  affiliations_list = data["Affiliations"].to_list()
  i = 0
  n_list = []
  for elem in data_list:
    affiliations_split = affiliations_list[i].split("; ")
    elem_split = elem.split("; ")
    omstu_list = []
    total = 0
    all_list = []
    for item in elem_split:
      item_split = item.split(' ')
      for affil in affiliations_split:
        if item.find(affil) != -1:
          if item.find("Omsk State Technical University") != -1:
            author = re.sub(r'[^A-Za-z]', '', item_split[0])
            omstu_list.append(author)
            all_list.append(author)
          else:
            author = re.sub(r'[^A-Za-z]', '', item_split[0])
            all_list.append(author)
    all_dict = dict(Counter(all_list))
    n2 = len(all_dict.keys())
    for item in omstu_list:
      total += 1 / all_dict[item]
    n = total / n2
    n_list.append(n)
    i += 1
  data["N"] = n_list
  return data

In [12]:
scopus_df = pd.read_excel("scopus2019.xlsx")
s1_df = pd.read_excel("s1.xlsx")
s2_df = pd.read_excel("s2.xlsx")
s3_df = pd.read_excel("s3.xlsx")
s4_df = pd.read_excel("s4.xlsx")

scopus_df_update = scopus_df.filter(
        ["Authors", "Title", "Source Title", "Affiliations", "Authors with affiliations"])
scopus_df_and_key = source_title_update(scopus_df_update)
s1_df_and_key = source_title_update(s1_df)
s2_df_and_key = source_title_update(s2_df)
s3_df_and_key = source_title_update(s3_df)
s4_df_and_key = source_title_update(s4_df)

result_s1_df = get_result(s1_df_and_key, scopus_df_and_key)
result_s1_df = get_n(result_s1_df)

result_s2_df = get_result(s2_df_and_key, scopus_df_and_key)
result_s2_df = get_n(result_s2_df)

result_s3_df = get_result(s3_df_and_key, scopus_df_and_key)
result_s3_df = get_n(result_s3_df)

result_s4_df = get_result(s4_df_and_key, scopus_df_and_key)
result_s4_df = get_n(result_s4_df)

con_data = pd.concat([result_s1_df, result_s2_df, result_s3_df, result_s4_df])
con_data.drop(["KEY", "N"], axis=1, inplace=True)
scopus_df_update.drop(["KEY"], axis=1, inplace=True)
result_s_none_df = pd.concat([scopus_df_update, con_data]).drop_duplicates(keep=False)
result_s_none_df = result_s_none_df.drop_duplicates(subset=["Title"], keep=False)
result_s_none_df = get_n(result_s_none_df)

result_s1_df.to_excel("s1_result.xlsx", index=False)
result_s2_df.to_excel("s2_result.xlsx", index=False)
result_s3_df.to_excel("s3_result.xlsx", index=False)
result_s4_df.to_excel("s4_result.xlsx", index=False)
result_s_none_df.to_excel("s_none_result.xlsx", index=False)